In [3]:
!pip install gdal

  Using cached gdal-3.11.4.tar.gz (878 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build gdal


  error: subprocess-exited-with-error
  
  × Building wheel for gdal (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [151 lines of output]
      Using numpy 2.3.3
      C:\Users\colto\AppData\Local\Temp\pip-build-env-l0hyu7z4\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsWarning: `license` overwritten by `pyproject.toml`
        corresp(dist, value, root_dir)
      C:\Users\colto\AppData\Local\Temp\pip-build-env-l0hyu7z4\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsWarning: `extras_require` overwritten in `pyproject.toml` (optional-dependencies)
        corresp(dist, value, root_dir)
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-313\osgeo
      copying osgeo\gdal.py -> build\lib.win-amd64-cpython-313\osgeo
      copying osgeo\gdalconst.py -> build\lib.win-amd64-cpython-313\osgeo
      copying osgeo\gdalnumeric.py -> build\lib.win-a

In [4]:
"""
Simple Rasterio Mosaic Dataset Creator
Creates time-enabled mosaic rasters just like ArcGIS mosaic datasets
"""

import numpy as np
import geopandas as gpd
import rasterio
from rasterio.transform import from_bounds
from scipy.ndimage import gaussian_filter
from pathlib import Path
from datetime import datetime
from osgeo import gdal

def create_kernel_density(gdf, value_field, cell_size=0.005, bandwidth=0.25):
    """Simple kernel density calculation"""

    # Get bounds
    bounds = list(gdf.total_bounds)
    bounds[0] -= 0.5  # Add buffer
    bounds[1] -= 0.5
    bounds[2] += 0.5
    bounds[3] += 0.5

    # Create grid
    cols = int((bounds[2] - bounds[0]) / cell_size)
    rows = int((bounds[3] - bounds[1]) / cell_size)

    x = np.linspace(bounds[0], bounds[2], cols)
    y = np.linspace(bounds[1], bounds[3], rows)
    xx, yy = np.meshgrid(x, y)

    # Calculate density
    density = np.zeros((rows, cols))

    for idx, point in gdf.iterrows():
        px, py = point.geometry.x, point.geometry.y
        value = point[value_field] if value_field in gdf.columns else 1

        if np.isnan(value):
            continue

        # Gaussian kernel
        dist = np.sqrt((xx - px)**2 + (yy - py)**2)
        kernel = np.exp(-dist**2 / (2 * bandwidth**2))
        density += kernel * value

    # Smooth and scale
    density = gaussian_filter(density, sigma=2) * 1000

    # Create transform
    transform = from_bounds(bounds[0], bounds[1], bounds[2], bounds[3], cols, rows)

    return density, transform, (rows, cols)


def process_to_mosaic(shapefile_path, output_dir, value_field='tweets'):
    """
    Process shapefile and create mosaic dataset

    Returns paths to created mosaic files
    """

    print(f"\nProcessing: {shapefile_path}")

    # Load data
    gdf = gpd.read_file(shapefile_path)
    if gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

    base_name = Path(shapefile_path).stem

    # Get timestamps
    if 'time_str' not in gdf.columns:
        print("  No time_str field found")
        return None

    timestamps = sorted(gdf['time_str'].unique())
    print(f"  Found {len(timestamps)} timestamps")

    # Process each timestamp
    raster_files = []
    cumulative = None

    for idx, timestamp in enumerate(timestamps):
        print(f"    [{idx+1}/{len(timestamps)}] {timestamp}")

        # Filter data
        time_gdf = gdf[gdf['time_str'] == timestamp]

        if len(time_gdf) == 0:
            continue

        # Create kernel density
        density, transform, shape = create_kernel_density(time_gdf, value_field)

        # Save raster
        safe_time = timestamp.replace(':', '-').replace(' ', '_')
        raster_path = output_dir / f"{base_name}_{idx:04d}_{safe_time}.tif"

        with rasterio.open(
            raster_path,
            'w',
            driver='GTiff',
            height=shape[0],
            width=shape[1],
            count=1,
            dtype='float32',
            crs='EPSG:4326',
            transform=transform,
            compress='lzw'
        ) as dst:
            dst.write(density.astype('float32'), 1)
            # Add timestamp metadata
            dst.update_tags(TIMESTAMP=timestamp, INDEX=idx)

        raster_files.append(str(raster_path))

        # Update cumulative
        if cumulative is None:
            cumulative = density.copy()
        else:
            cumulative += density

        # Save cumulative
        cumul_path = output_dir / f"{base_name}_cumulative_{idx:04d}.tif"
        with rasterio.open(
            cumul_path,
            'w',
            driver='GTiff',
            height=shape[0],
            width=shape[1],
            count=1,
            dtype='float32',
            crs='EPSG:4326',
            transform=transform,
            compress='lzw'
        ) as dst:
            dst.write(cumulative.astype('float32'), 1)
            dst.update_tags(TIMESTAMP=timestamp, INDEX=idx, TYPE='cumulative')

    # Create mosaic datasets
    if raster_files:
        mosaic_files = create_mosaic_datasets(raster_files, output_dir, base_name, timestamps)
        return mosaic_files

    return None


def create_mosaic_datasets(raster_files, output_dir, base_name, timestamps):
    """
    Create multiple types of mosaic datasets
    """

    print(f"\n  Creating mosaic datasets...")

    outputs = {}

    # 1. VRT Mosaic (Virtual Dataset - most efficient)
    vrt_path = output_dir / f"{base_name}_mosaic.vrt"
    vrt = gdal.BuildVRT(str(vrt_path), raster_files, separate=True)

    if vrt:
        # Add band metadata
        for i, (raster, timestamp) in enumerate(zip(raster_files, timestamps), 1):
            band = vrt.GetRasterBand(i)
            band.SetDescription(timestamp)

            # Parse datetime
            try:
                dt = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                band.SetMetadataItem('TIMESTAMP', dt.isoformat())
            except:
                band.SetMetadataItem('TIMESTAMP', timestamp)

        vrt.FlushCache()
        vrt = None
        outputs['vrt'] = str(vrt_path)
        print(f"    ✅ VRT mosaic: {vrt_path.name}")

    # 2. Multi-band GeoTIFF (all timestamps in one file)
    multiband_path = output_dir / f"{base_name}_multiband.tif"

    # Read first raster for template
    with rasterio.open(raster_files[0]) as src:
        profile = src.profile.copy()
        profile.update(count=len(raster_files))

        # Create multi-band raster
        with rasterio.open(multiband_path, 'w', **profile) as dst:
            for band_idx, raster_file in enumerate(raster_files, 1):
                with rasterio.open(raster_file) as band_src:
                    dst.write(band_src.read(1), band_idx)
                    dst.set_band_description(band_idx, timestamps[band_idx-1])

            # Add metadata
            dst.update_tags(
                TEMPORAL='true',
                START_TIME=timestamps[0],
                END_TIME=timestamps[-1],
                TIME_STEPS=str(len(timestamps))
            )

    outputs['multiband'] = str(multiband_path)
    print(f"    ✅ Multi-band raster: {multiband_path.name}")

    # 3. Create time metadata file for GIS software
    metadata_path = output_dir / f"{base_name}_time_metadata.txt"
    with open(metadata_path, 'w') as f:
        f.write("Mosaic Dataset Time Information\n")
        f.write("=" * 40 + "\n")
        f.write(f"Total time slices: {len(timestamps)}\n")
        f.write(f"Start: {timestamps[0]}\n")
        f.write(f"End: {timestamps[-1]}\n")
        f.write("\nBand mappings:\n")
        for i, ts in enumerate(timestamps, 1):
            f.write(f"  Band {i}: {ts}\n")

    outputs['metadata'] = str(metadata_path)

    return outputs


def main():
    """Main processing"""

    # Configuration - UPDATE THESE PATHS
    input_gdb = Path(r"C:\Users\clgoodr2\Documents\ArcGIS\Projects\MyProject\MyProject.gdb")
    output_dir = Path(r"C:\Users\clgoodr2\Documents\ArcGIS\Projects\MyProject\Rasterio_Mosaics")

    output_dir.mkdir(parents=True, exist_ok=True)

    # Process jobs
    jobs = {
        "francine_cities_freq_over_time_points": "tweets",
        "francine_cities_cumulative_points": "cum",
        "helene_cities_freq_over_time": "tweets",
        "helene_cities_cumulative": "cum"
    }

    results = {}

    for shapefile, field in jobs.items():
        # Try with and without .shp extension
        shp_path = input_gdb / f"{shapefile}.shp"
        if not shp_path.exists():
            shp_path = input_gdb / shapefile

        if shp_path.exists():
            mosaic_files = process_to_mosaic(shp_path, output_dir, field)
            if mosaic_files:
                results[shapefile] = mosaic_files
        else:
            print(f"\n⚠️ Not found: {shapefile}")

    # Summary
    print("\n" + "="*60)
    print("MOSAIC DATASETS CREATED!")
    print("="*60)

    for source, files in results.items():
        print(f"\n{source}:")
        print(f"  - VRT Mosaic: {Path(files['vrt']).name}")
        print(f"  - Multi-band TIFF: {Path(files['multiband']).name}")

    print("\n" + "="*60)
    print("HOW TO USE THE MOSAIC DATASETS:")
    print("="*60)

    print("\n🗺️ IN ARCGIS PRO:")
    print("1. Add the _multiband.tif file to your map")
    print("2. It will show as a multi-band raster")
    print("3. Use Image Analysis tools to work with bands")
    print("4. Or use Make Multidimensional Raster tool")
    print("5. Configure time properties and animate")

    print("\n🗺️ IN QGIS (FREE ALTERNATIVE):")
    print("1. Add the .vrt file to QGIS")
    print("2. Right-click layer → Properties → Temporal")
    print("3. Set Configuration to 'Fixed Time Range Per Band'")
    print("4. Open Temporal Controller panel")
    print("5. Use time slider to animate through bands")

    print("\n🗺️ IN PYTHON:")
    print("import rasterio")
    print("with rasterio.open('your_multiband.tif') as src:")
    print("    for band in range(1, src.count + 1):")
    print("        data = src.read(band)")
    print("        # Process each timestamp...")


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'osgeo'